In [1]:
import pandas as pd
import numpy as np
import sklearn
from sklearn import preprocessing

import keras
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.callbacks import ModelCheckpoint
import re
import os
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import median_absolute_error
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split

Using TensorFlow backend.


In [2]:
#the function finds best epoch
def best_epoch_finder(path2weights):
    best_epoch = max([int(x.split('-')[2]) for x in os.listdir(path2weights) if x[len(x)-4:len(x)] == 'hdf5'])
    out_filename = [np.sort(os.listdir(path2weights))[i] for i in range(0, len(os.listdir(path2weights))) if int(re.split('-', np.sort(os.listdir(path2weights))[i])[2]) == best_epoch][0]

    #del all garbage
    dustbin_list = os.listdir(path2weights)
    dustbin_list.remove(out_filename)
    
    [os.remove(path2weights + i) for i in dustbin_list]
    #del done!

    return(best_epoch, out_filename)

In [6]:
rand_split = [[42,12],[23,5],[732,19]] # Random states to split data into training/validation/test sets
#neur_in_lay = [512,256,128,64,32,16] # Neurons in layers for perceptron with 1 hidden layer
#neur_in_lay = [[750,256],[512,128],[128,32],[64,16],[32,8]] # Neurons in layers for perceptron with 2 hidden layers
neur_in_lay = [[512,128,32],[128,64,32],[64,32,16],[32,16,8]] # Neurons in layers for perceptron with 3 hidden layers

In [7]:
gen_path = 'Desktop/Gala_Salts/' #Path to the data
file_name = 'FL_350.csv' # File with data
y_ion_file = 'Y_ions.csv' #File with true outputs


for splt in rand_split: # Here we iterate through cross-validation splits
    
    os.makedirs(gen_path +'split_' + str(splt[0]) + '_' +  str(splt[1]) +'/', exist_ok = True) 
    path = gen_path + 'split_' + str(splt[0]) + '_' +  str(splt[1]) +'/' 

    #Data import
    Data = pd.read_csv(gen_path + file_name, sep=',') 
    data = np.array(Data) 
    
    Y_ions = pd.read_csv(gen_path + 'Y_ions.csv', sep=';')
    
    # Define X, Y
    Y = np.array(Y_ions)[:,1:] 
    X = data[:,1:]
    
    # Split data into train, validation and test sets
    X_trn, X_30, Y_trn, Y_30 = train_test_split(X, Y, test_size=0.3, random_state=splt[0])
    X_tst, X_pro, Y_tst, Y_pro = train_test_split(X_30, Y_30, test_size = 0.3333, random_state=splt[1])

    # Write splitted X data into file
    pd.DataFrame(X_trn).to_csv(path + 'X_trn.csv', sep=',', header = list(Data.columns.values[1:]))
    pd.DataFrame(X_tst).to_csv(path + 'X_tst.csv', sep=',', header = list(Data.columns.values[1:]))
    pd.DataFrame(X_pro).to_csv(path + 'X_pro.csv', sep=',', header = list(Data.columns.values[1:]))

    #Write splitted Y data into file
    a = ['Cu','Ni','Cr','No3']
        
    pd.DataFrame(Y_trn).to_csv(path + 'Y_trn.csv', sep=',', header = a)
    pd.DataFrame(Y_tst).to_csv(path + 'Y_tst.csv', sep=',', header = a)
    pd.DataFrame(Y_pro).to_csv(path + 'Y_pro.csv', sep=',', header = a)

    #Data Scaling

    scalerX = preprocessing.MinMaxScaler()
    scalerY = preprocessing.MinMaxScaler()

    x_trn = scalerX.fit_transform(X_trn)
    x_tst = scalerX.transform(X_tst)
    x_pro = scalerX.transform(X_pro)

    y_trn = scalerY.fit_transform(Y_trn)
    y_tst = scalerY.transform(Y_tst)
    y_pro = scalerY.transform(Y_pro)

    for numb_neur in neur_in_lay: # Here we iterate through model's parameters (nuber of neurons in hidden layers)
        os.makedirs(path + str(numb_neur) + '/',exist_ok = True)

        loc_path = path + str(numb_neur) + '/'

        for i in range(0,3): # Three initializatins of the network
            os.makedirs(loc_path+str(i)+ '/',exist_ok = True)

            # Create model
            model = Sequential()
            model.add(Dense(numb_neur[0], input_dim=x_trn.shape[1], kernel_initializer='glorot_uniform', activation='sigmoid'))
            model.add(Dropout(0.45))
            model.add(Dense(numb_neur[1], kernel_initializer='glorot_uniform', activation='sigmoid'))
            model.add(Dropout(0.45))
            model.add(Dense(numb_neur[2], kernel_initializer='glorot_uniform', activation='sigmoid'))
            model.add(Dropout(0.45))
            model.add(Dense(y_trn.shape[1], kernel_initializer='glorot_uniform', activation='linear', use_bias=True))

            path2model = loc_path+str(i)+ '/'

            model.compile(loss='mean_squared_error', optimizer='adagrad', metrics=['mae'])

            filename = 'weights-improvement-{epoch:02d}-{val_loss:.5f}.hdf5'
            checkpoint = ModelCheckpoint(loc_path+str(i)+ '/'+filename, monitor='val_loss', verbose=1, save_best_only=True, mode='min')
            callbacks_list = [checkpoint]
            
            #Training
            tst_stop = 500
                
            for epoch_step in range(0, 100000, tst_stop):
                if (epoch_step!=0):
                    last_best_epoch, best_weights_file = best_epoch_finder(path2model)
                    print(last_best_epoch, epoch_step)
                    if ((tst_stop+int(last_best_epoch))>epoch_step):
                        model.load_weights(path2model+best_weights_file)
                        model.compile(loss='mean_squared_error', optimizer='adagrad', metrics=['mae'])
                        model.fit(x_trn, y_trn, epochs=epoch_step+tst_stop, batch_size=64,  verbose=0, callbacks=callbacks_list, shuffle=True,validation_data = (x_tst, y_tst), initial_epoch=epoch_step)
                    else:
                        continue
                if (epoch_step==0):
                    model.fit(x_trn, y_trn, epochs=epoch_step+tst_stop, batch_size=64,  verbose=0, callbacks=callbacks_list, shuffle=True,validation_data = (x_tst, y_tst))

            best_epoch_finder(path2model)

            model_json = model.to_json()
            with open(path2model + 'FL_350_model'+'.json', 'w') as json_file: json_file.write(model_json)

            y_predicted_trn = scalerY.inverse_transform(model.predict(x_trn))
            y_predicted_trn[y_predicted_trn<0]=0

            y_predicted_tst = scalerY.inverse_transform(model.predict(x_tst))
            y_predicted_tst[y_predicted_tst<0]=0

            y_predicted_pro = scalerY.inverse_transform(model.predict(x_pro))
            y_predicted_pro[y_predicted_pro<0]=0

            pd.DataFrame(y_predicted_trn).to_csv(loc_path + str(i) + '/' + 'y_pred_trn.csv', sep=',')             
            pd.DataFrame(y_predicted_tst).to_csv(loc_path + str(i)+ '/' + 'y_pred_tst.csv', sep=',')
            pd.DataFrame(y_predicted_pro).to_csv(loc_path + str(i)+ '/' + 'y_pred_pro.csv', sep=',')
    
        #Statistics

        mae_trn = np.zeros((5,4))
        mae_tst = np.zeros((5,4))
        mae_pro = np.zeros((5,4))

        mse_trn = np.zeros((5,4))
        mse_tst = np.zeros((5,4))
        mse_pro = np.zeros((5,4))

        r2_trn = np.zeros((5,4))
        r2_tst = np.zeros((5,4))
        r2_pro = np.zeros((5,4))

        for j in range(0,3):
            y_pred_trn = np.array(pd.read_csv(loc_path + str(j) + '/y_pred_trn.csv'))[:,1:]
            y_pred_tst = np.array(pd.read_csv(loc_path + str(j) + '/y_pred_tst.csv'))[:,1:]
            y_pred_pro = np.array(pd.read_csv(loc_path + str(j) + '/y_pred_pro.csv'))[:,1:]

            #MAE
            mae_trn[j,:] = mean_absolute_error(np.array(Y_trn), y_pred_trn, multioutput='raw_values')
            mae_tst[j,:] = mean_absolute_error(np.array(Y_tst), y_pred_tst, multioutput='raw_values')
            mae_pro[j,:] = mean_absolute_error(np.array(Y_pro), y_pred_pro, multioutput='raw_values')

            #MSE
            mse_trn[j,:] = mean_squared_error(np.array(Y_trn), y_pred_trn, multioutput='raw_values')
            mse_tst[j,:] = mean_squared_error(np.array(Y_tst), y_pred_tst, multioutput='raw_values')
            mse_pro[j,:] = mean_squared_error(np.array(Y_pro), y_pred_pro, multioutput='raw_values')

            #R^2
            r2_trn[j,:] = r2_score(np.array(Y_trn), y_pred_trn, multioutput='raw_values')
            r2_tst[j,:] = r2_score(np.array(Y_tst), y_pred_tst, multioutput='raw_values')
            r2_pro[j,:] = r2_score(np.array(Y_pro), y_pred_pro, multioutput='raw_values')   


        pd.DataFrame((np.mean(mae_trn, axis=0)).reshape(1,len(np.mean(mae_trn, axis=0)))).to_csv(loc_path + 'MAE_trn.csv', sep=',', header = a)
        pd.DataFrame((np.mean(mae_tst, axis=0)).reshape(1,len(np.mean(mae_tst, axis=0)))).to_csv(loc_path + 'MAE_tst.csv', sep=',', header = a)
        pd.DataFrame((np.mean(mae_pro, axis=0)).reshape(1,len(np.mean(mae_pro, axis=0)))).to_csv(loc_path + 'MAE_pro.csv', sep=',', header = a)

        pd.DataFrame((np.mean(mse_trn, axis=0)).reshape(1,len(np.mean(mse_trn, axis=0)))).to_csv(loc_path + 'MSE_trn.csv', sep=',', header = a)
        pd.DataFrame((np.mean(mse_tst, axis=0)).reshape(1,len(np.mean(mse_tst, axis=0)))).to_csv(loc_path + 'MSE_tst.csv', sep=',', header = a)
        pd.DataFrame((np.mean(mse_pro, axis=0)).reshape(1,len(np.mean(mse_pro, axis=0)))).to_csv(loc_path + 'MSE_pro.csv', sep=',', header = a)

        pd.DataFrame((np.mean(r2_trn, axis=0)).reshape(1,len(np.mean(r2_trn, axis=0)))).to_csv(loc_path + 'R2_trn.csv', sep=',', header = a)
        pd.DataFrame((np.mean(r2_tst, axis=0)).reshape(1,len(np.mean(r2_tst, axis=0)))).to_csv(loc_path + 'R2_tst.csv', sep=',', header = a)
        pd.DataFrame((np.mean(r2_pro, axis=0)).reshape(1,len(np.mean(r2_pro, axis=0)))).to_csv(loc_path + 'R2_pro.csv', sep=',', header = a)


(1000, 500)

Epoch 00001: val_loss improved from inf to 0.09605, saving model to Desktop/Gala_Salts/split_42_12/[128, 64, 32]/0/weights-improvement-01-0.09605.hdf5

Epoch 00002: val_loss did not improve from 0.09605

Epoch 00003: val_loss did not improve from 0.09605

Epoch 00004: val_loss did not improve from 0.09605

Epoch 00005: val_loss did not improve from 0.09605

Epoch 00006: val_loss did not improve from 0.09605

Epoch 00007: val_loss did not improve from 0.09605

Epoch 00008: val_loss did not improve from 0.09605

Epoch 00009: val_loss did not improve from 0.09605

Epoch 00010: val_loss did not improve from 0.09605

Epoch 00011: val_loss did not improve from 0.09605

Epoch 00012: val_loss did not improve from 0.09605

Epoch 00013: val_loss did not improve from 0.09605

Epoch 00014: val_loss did not improve from 0.09605

Epoch 00015: val_loss did not improve from 0.09605

Epoch 00016: val_loss did not improve from 0.09605

Epoch 00017: val_loss improved from 0.09605 to 0.09577,